In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
NAME = "intl"

## Make Companies CSV

In [3]:
cols = ['CompanyName', 'twitterUrl', 'linkedInUrl', 'facebookUrl', 'websiteUrl', \
        'city', 'country', 'countryIso', 'state', 'locationString', 'isPrimary', \
        'employeeSizeRange', 'description']
company_df_dict = {colname: [] for colname in cols}
count_lines = 0

for filepath in ["../data/TO_orgcharts_CN.json", "../data/TO_orgcharts_CA.json", "../data/TO_orgcharts_IN.json"]: 
    f = open(filepath)
    data = json.load(f)
    for line in data['orgCharts']:
        count_lines += 1
        company_df_dict['CompanyName'].append(line['CompanyName'])

        company_df_dict['twitterUrl'].append(line['stats']['social']['twitterUrl'])
        company_df_dict['linkedInUrl'].append(line['stats']['social']['linkedInUrl'])
        company_df_dict['facebookUrl'].append(line['stats']['social']['facebookUrl'])
        company_df_dict['websiteUrl'].append(line['stats']['social']['websiteUrl'])

        company_df_dict['city'].append(line['stats']['location']['city'])
        company_df_dict['country'].append(line['stats']['location']['country'])
        company_df_dict['countryIso'].append(line['stats']['location']['countryIso'])
        company_df_dict['state'].append(line['stats']['location']['state'])
        company_df_dict['locationString'].append(line['stats']['location']['locationString'])
        company_df_dict['isPrimary'].append(line['stats']['location']['isPrimary'])

        company_df_dict['employeeSizeRange'].append(line['stats']['employeeRange'])

        company_df_dict['description'].append(line['stats']['description'])
    f.close()
    print(count_lines)

print(count_lines)
company_df = pd.DataFrame.from_dict(company_df_dict)

213
3089
5692
5692


In [4]:
company_df

,CompanyName,twitterUrl,linkedInUrl,facebookUrl,websiteUrl,city,country,countryIso,state,locationString,isPrimary,employeeSizeRange,description
0,incoPat,https://twitter.com/incoshare,https://www.linkedin.com/company/beijing-incop...,None,http://www.incopat.com,Haidian,China,CN,None,"Haidian, Beijing, China",True,200-500,incoPat is a technical innovative intelligence...
1,Pactics,None,https://www.linkedin.com/company/pactics-shang...,https://www.facebook.com/pactics.kh,https://www.pactics.com/,Shanghai,China,CN,Shanghai Shi,None,True,200-500,Pactics creates high-quality microfiber access...
2,Impactiva,None,https://www.linkedin.com/company/impactiva-s-d...,https://www.facebook.com/impactiva,https://impactiva.com/,Dongguan City,China,CN,Guangdong Sheng,None,True,200-500,Impactiva is the world's leader in Process Opt...
3,Keep,None,https://www.linkedin.com/company/beijing-calor...,https://www.facebook.com/KeepTrainer/,http://www.gotokeep.com/,Beijing,China,CN,None,"Beijing, Beijing, China",True,200-500,Keep is a mobile fitness community that offers...
4,SuperOrdinary,None,https://linkedin.com/company/superordinaryco,https://www.facebook.com/SuperOrdinaryCo,http://superordinary.co,Shanghai,China,CN,Shanghai,None,True,200-500,SuperOrdinary is the leading global growth par...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5687,Gsquare Web Technologies,https://twitter.com/gsquarewebtech,https://www.linkedin.com/company/gsquare-web-t...,https://www.facebook.com/gsquarewebtechnologies,https://www.gsquarewebtech.com/,Mohali,India,IN,None,"Mohali, Punjab, India",True,10-50,"Hire Mobile App, Custom web app team & digital..."
5688,WeddingWishlist,https://twitter.com/weddingwishlst,None,https://www.facebook.com/weddingwishlistonline/,https://www.weddingwishlist.com/,Chennai,India,IN,None,"Chennai, Tamil Nadu, India",True,10-50,Create a wishlist and help your guests choose ...
5689,Simnovus,https://www.twitter.com/simnovustech,https://linkedin.com/company/simnovus-tech,https://www.facebook.com/Simnovus-Tech-6806069...,https://simnovus.com,New Delhi,India,IN,DL,"New Delhi, Delhi, India",True,10-50,Simnovus is a young test and measurement compa...
5690,String Bio,None,https://www.linkedin.com/company/string-bio-pv...,None,http://www.stringbio.com/,Bangalore,India,IN,None,"Bangalore, Karnataka, India",True,10-50,String Bio designs sustainable industrial prod...


In [5]:
company_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5692 entries, 0 to 5691
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CompanyName        5692 non-null   object
 1   twitterUrl         4527 non-null   object
 2   linkedInUrl        5425 non-null   object
 3   facebookUrl        4171 non-null   object
 4   websiteUrl         5692 non-null   object
 5   city               5691 non-null   object
 6   country            5692 non-null   object
 7   countryIso         5692 non-null   object
 8   state              2425 non-null   object
 9   locationString     4048 non-null   object
 10  isPrimary          5692 non-null   bool  
 11  employeeSizeRange  5692 non-null   object
 12  description        5664 non-null   object
dtypes: bool(1), object(12)
memory usage: 539.3+ KB


In [6]:
company_df['employeeSizeRange'].value_counts()

10-50      2784
50-200     1691
200-500    1217
Name: employeeSizeRange, dtype: int64

### Clean locations - lots of missing states & cities but some info is in locationString

"Shi" means city, "Sheng" means state in Chinese so strip these to get just city/state names by themselves

In [7]:
def parse_city(row): 
    if row['city'] is not None: 
        city = row['city']
        city = city.replace(" Shi", "")
        city = city.replace(" Sheng", "")
        city = city.replace(" City", "")
        return city
    elif row['locationString'] is not None: 
        x = row['locationString'].split(',')
        print(x)
        if len(x) == 3: 
            city = x[0]
            city = city.replace(" Shi", "")
            city = city.replace(" Sheng", "")
            city = city.replace(" City", "")
            if city[-1] == ' ': 
                return city[-1]
            else: 
                return city
    return ''

def parse_state(row): 
    if row['state'] is not None: 
        state = row['state']
        state = state.replace(" Shi", "")
        state = state.replace(" Sheng", "")
        state = state.replace(" City", "")
        return state
    elif row['locationString'] is not None: 
        x = row['locationString'].split(',')
        if len(x) == 3: 
            state = x[1]
            state = state.replace(" Shi", "")
            state = state.replace(" Sheng", "")
            state = state.replace(" City", "")
            if state[-1] == ' ': 
                return state[-1]
            else: 
                return state
    return ''

def parse_country(row): 
    if row['country'] is not None: 
        return row['country']
    elif row['locationString'] is not None: 
        x = row['locationString'].split(',')
        if len(x) == 3: 
            country = x[2]
            if country[-1] == ' ': 
                return country[-1]
            else: 
                return country
    return ''

In [8]:
company_df['city'] = company_df.apply(lambda row: parse_city(row), axis=1)
company_df['state'] = company_df.apply(lambda row: parse_state(row), axis=1)
company_df['country'] = company_df.apply(lambda row: parse_country(row), axis=1)
company_df['locationString'] = company_df.apply(lambda row: row["city"] + ", " + row["state"] + ", " + row["country"], axis=1)

In [9]:
company_df[['city', 'state', 'country', 'locationString']]

,city,state,country,locationString
0,Haidian,Beijing,China,"Haidian, Beijing, China"
1,Shanghai,Shanghai,China,"Shanghai, Shanghai, China"
2,Dongguan,Guangdong,China,"Dongguan, Guangdong, China"
3,Beijing,Beijing,China,"Beijing, Beijing, China"
4,Shanghai,Shanghai,China,"Shanghai, Shanghai, China"
...,...,...,...,...
5687,Mohali,Punjab,India,"Mohali, Punjab, India"
5688,Chennai,Tamil Nadu,India,"Chennai, Tamil Nadu, India"
5689,New Delhi,DL,India,"New Delhi, DL, India"
5690,Bangalore,Karnataka,India,"Bangalore, Karnataka, India"


### Check # Companies

In [10]:
company_df["CompanyName"].value_counts()

MobileCoderz technologies Pvt. Ltd.    22
ITEK Solutions                         21
SensorUp                               10
Osperity Inc.                          10
Algoscale Technologies, Inc.           10
                                       ..
Business Instincts Group                1
Reveal Marketing Group                  1
The Very Good Butchers                  1
Sport for Life                          1
Simnovus                                1
Name: CompanyName, Length: 3443, dtype: int64

In [11]:
company_df.columns

Index(['CompanyName', 'twitterUrl', 'linkedInUrl', 'facebookUrl', 'websiteUrl',
       'city', 'country', 'countryIso', 'state', 'locationString', 'isPrimary',
       'employeeSizeRange', 'description'],
      dtype='object')

### Drop unnecessary columns

In [12]:
company_df.drop(columns=['countryIso', 'isPrimary'], inplace=True)

In [13]:
len(company_df["CompanyName"].value_counts())

3443

### Drop duplicate companies & save companies_df

In [14]:
company_df.drop_duplicates(subset="CompanyName", inplace=True)

# Save
company_df.to_csv(f"companies_{NAME}.csv")

## Make Org Chart CSV

In [68]:
cols = ['CompanyName', 'id', 'title', 'leafMember', 'containingNodeId', 'order', 'parentId', 'section', 'type']
orgchart_df_dict = {colname: [] for colname in cols}
count_lines = 0

for filepath in ["data/TO_orgcharts_CN.json", "data/TO_orgcharts_CA.json", "data/TO_orgcharts_IN.json"]: 
    f = open(filepath)
    data = json.load(f)
    for line in data['orgCharts']:
        count_lines += 1
        employeeNodes = line['employeeNodes']
        for i in range(len(employeeNodes)):
            if line['employeeNodes'][i]['__typename'] == 'OrgChartStructureNode': 

                for col in cols: 
                    if col == 'CompanyName': 
                        orgchart_df_dict['CompanyName'].append(line['CompanyName'])
                    else: 
                        orgchart_df_dict[col].append(line['employeeNodes'][i][col])
            # Print type if not OrgChartStructureNode 
            else: 
                print(line['employeeNodes'][i]['__typename'])
    print(count_lines)
            
orgchart_df = pd.DataFrame.from_dict(orgchart_df_dict)
orgchart_df

213
3089
5692


,CompanyName,id,title,leafMember,containingNodeId,order,parentId,section,type
0,incoPat,p-1439351,Amanda Wang,"{'id': 1439351, 'slug': 'amanda-wang', 'fullNa...",None,2,p-1439346,orgChart,leaf
1,incoPat,p-1439347,Waylen Chen,"{'id': 1439347, 'slug': 'waylen-chen', 'fullNa...",None,1,p-1439346,orgChart,leaf
2,incoPat,p-1439348,Tingxin Jiang,"{'id': 1439348, 'slug': 'tingxin-jiang', 'full...",None,0,p-1439346,orgChart,leaf
3,incoPat,p-1439346,Jason Wu,"{'id': 1439346, 'slug': 'jason-wu', 'fullName'...",None,0,None,orgChart,leaf
4,Pactics,p-1198798,Rose Orm,"{'id': 1198798, 'slug': 'rose-orm', 'fullName'...",None,0,p-1198795,orgChart,leaf
...,...,...,...,...,...,...,...,...,...
70330,SectorQube,p-1679524,Sabarish Prakash,"{'id': 1679524, 'slug': 'sabarish-prakash', 'f...",None,4,p-1679503,orgChart,leaf
70331,SectorQube,p-1679522,Biniyas Valiyaveettil,"{'id': 1679522, 'slug': 'biniyas-valiyaveettil...",None,3,p-1679503,orgChart,leaf
70332,SectorQube,p-1679515,Arjun Sarath,"{'id': 1679515, 'slug': 'arjun-sarath', 'fullN...",None,2,p-1679503,orgChart,leaf
70333,SectorQube,p-1679512,Ani Abraham Joy,"{'id': 1679512, 'slug': 'ani-abraham-joy', 'fu...",None,1,p-1679503,orgChart,leaf


### Clean IDs, names

In [69]:
# Clean id's to not include "p-"
orgchart_df["id"] = orgchart_df["id"].apply(lambda x: None if x is None else int(x[2:]) if x[0:2] == 'p-' else x)
orgchart_df["parentId"] = orgchart_df["parentId"].apply(lambda x: None if x is None else int(x[2:]) if x[0:2] == 'p-' else x)

# Rename title to name
orgchart_df.rename(columns={"id": "employeeId", "title": "employeeName"}, inplace=True)

In [70]:
orgchart_df

,CompanyName,employeeId,employeeName,leafMember,containingNodeId,order,parentId,section,type
0,incoPat,1439351,Amanda Wang,"{'id': 1439351, 'slug': 'amanda-wang', 'fullNa...",None,2,1439346,orgChart,leaf
1,incoPat,1439347,Waylen Chen,"{'id': 1439347, 'slug': 'waylen-chen', 'fullNa...",None,1,1439346,orgChart,leaf
2,incoPat,1439348,Tingxin Jiang,"{'id': 1439348, 'slug': 'tingxin-jiang', 'full...",None,0,1439346,orgChart,leaf
3,incoPat,1439346,Jason Wu,"{'id': 1439346, 'slug': 'jason-wu', 'fullName'...",None,0,None,orgChart,leaf
4,Pactics,1198798,Rose Orm,"{'id': 1198798, 'slug': 'rose-orm', 'fullName'...",None,0,1198795,orgChart,leaf
...,...,...,...,...,...,...,...,...,...
70330,SectorQube,1679524,Sabarish Prakash,"{'id': 1679524, 'slug': 'sabarish-prakash', 'f...",None,4,1679503,orgChart,leaf
70331,SectorQube,1679522,Biniyas Valiyaveettil,"{'id': 1679522, 'slug': 'biniyas-valiyaveettil...",None,3,1679503,orgChart,leaf
70332,SectorQube,1679515,Arjun Sarath,"{'id': 1679515, 'slug': 'arjun-sarath', 'fullN...",None,2,1679503,orgChart,leaf
70333,SectorQube,1679512,Ani Abraham Joy,"{'id': 1679512, 'slug': 'ani-abraham-joy', 'fu...",None,1,1679503,orgChart,leaf


## Parse LeafMembers

In [72]:
orgchart_df.iloc[70334]["leafMember"]

{'id': 1679503,
 'slug': 'nibu-alias',
 'fullName': 'Nibu Alias',
 'role': 'Co-Founder & CEO',
 'roleFunction': None,
 'roleAutoFunction': 'admn',
 'description': '',
 'parentPositionId': None,
 'profileImage': {'endpoint': 'https://cdn.theorg.com',
  'ext': 'jpg',
  'placeholderDataUrl': None,
  'prevailingColor': '#ffffff',
  'uri': '2e65e9e5-3244-4ef2-b5e1-1ec4e7950d62',
  'versions': ['thumb', 'medium', 'small', 'original', 'large'],
  '__typename': 'Image'},
 'social': {'twitterUrl': None,
  'linkedInUrl': 'https://www.linkedin.com/in/nibu-alias-09b36164/',
  'facebookUrl': 'https://www.facebook.com/nibu.alias',
  'websiteUrl': None,
  '__typename': 'UserSocial'},
 'isAdviser': False,
 'group': None,
 'companyStartDate': None,
 'roleStartDate': None,
 'location': None,
 'invitedAt': None,
 'remote': False,
 'lastUpdate': '2022-08-18T16:04:44.587',
 'pronoun': None,
 'claimedBy': None,
 '__typename': 'FlatPosition'}

In [73]:
cols = ["slug", "role", "roleFunction", "roleAutoFunction", "description", "parentPositionId", "isAdviser", "group", 
       "companyStartDate", "roleStartDate", "location", "invitedAt", "remote", "lastUpdate", "pronoun", "claimedBy"]
for colname in cols:
    orgchart_df[colname] = orgchart_df["leafMember"].apply(lambda leaf_dict: leaf_dict[colname] if leaf_dict is not None and colname in leaf_dict.keys() else None)
    
social_cols = ["twitterUrl", "linkedInUrl", "facebookUrl", "websiteUrl"]
for colname in social_cols:
    orgchart_df[colname] = orgchart_df["leafMember"].apply(lambda leaf_dict: leaf_dict['social'][colname] if leaf_dict is not None and colname in leaf_dict.keys() else None)

cols_to_drop = ['leafMember', 'containingNodeId', 'roleFunction', 'group', 'companyStartDate', 'roleStartDate', 
               'location', 'invitedAt', 'pronoun', 'claimedBy', 'section'] + social_cols
orgchart_df.drop(columns=cols_to_drop, inplace=True)

In [74]:
orgchart_df

,CompanyName,employeeId,employeeName,order,parentId,type,slug,role,roleAutoFunction,description,parentPositionId,isAdviser,remote,lastUpdate
0,incoPat,1439351,Amanda Wang,2,1439346,leaf,amanda-wang,VP,sale,Master's degree in economics of Renmin Univers...,1439346.0,False,False,2022-08-27T04:36:45.107
1,incoPat,1439347,Waylen Chen,1,1439346,leaf,waylen-chen,VP,sale,Graduated from Shandong University majoring in...,1439346.0,False,False,2022-08-27T04:36:44.036
2,incoPat,1439348,Tingxin Jiang,0,1439346,leaf,tingxin-jiang,CTO,it,Master's degree of Beijing University of Aeron...,1439346.0,False,False,2022-08-27T04:42:47.354
3,incoPat,1439346,Jason Wu,0,None,leaf,jason-wu,CEO,admn,MBA of the University of North Carolina at Cha...,NaN,False,False,2022-08-27T04:32:44.135
4,Pactics,1198798,Rose Orm,0,1198795,leaf,rose-orm,Product Development Manager,prod,None,1198795.0,False,False,2022-04-13T16:39:53.975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70330,SectorQube,1679524,Sabarish Prakash,4,1679503,leaf,sabarish-prakash,Co-Founder & CBDO,admn,,1679503.0,False,False,2022-08-18T16:05:58.701
70331,SectorQube,1679522,Biniyas Valiyaveettil,3,1679503,leaf,biniyas-valiyaveettil,Co-Founder & CMO,mktg,Biniyas Valiyaveettil is the co-founder and CM...,1679503.0,False,False,2022-09-26T12:03:52.905
70332,SectorQube,1679515,Arjun Sarath,2,1679503,leaf,arjun-sarath,Co-Founder & CPO,admn,,1679503.0,False,False,2022-08-18T16:05:20.510
70333,SectorQube,1679512,Ani Abraham Joy,1,1679503,leaf,ani-abraham-joy,Co-Founder & COO,ops,,1679503.0,False,False,2022-08-18T16:05:08.408


### Create Employee table

In [75]:
employee_df = orgchart_df[['employeeId', 'slug', 'CompanyName', 'employeeName', 'order', 'role', 
                           'roleAutoFunction', 'description', 'remote']]
employee_df.set_index('employeeId', inplace=True)

In [76]:
employee_df

,slug,CompanyName,employeeName,order,role,roleAutoFunction,description,remote
employeeId,,,,,,,,
1439351,amanda-wang,incoPat,Amanda Wang,2,VP,sale,Master's degree in economics of Renmin Univers...,False
1439347,waylen-chen,incoPat,Waylen Chen,1,VP,sale,Graduated from Shandong University majoring in...,False
1439348,tingxin-jiang,incoPat,Tingxin Jiang,0,CTO,it,Master's degree of Beijing University of Aeron...,False
1439346,jason-wu,incoPat,Jason Wu,0,CEO,admn,MBA of the University of North Carolina at Cha...,False
1198798,rose-orm,Pactics,Rose Orm,0,Product Development Manager,prod,None,False
...,...,...,...,...,...,...,...,...
1679524,sabarish-prakash,SectorQube,Sabarish Prakash,4,Co-Founder & CBDO,admn,,False
1679522,biniyas-valiyaveettil,SectorQube,Biniyas Valiyaveettil,3,Co-Founder & CMO,mktg,Biniyas Valiyaveettil is the co-founder and CM...,False
1679515,arjun-sarath,SectorQube,Arjun Sarath,2,Co-Founder & CPO,admn,,False


In [77]:
employee_df.to_csv(f"employee_{NAME}.csv")

### Create orgchart table

In [78]:
chart_df = orgchart_df[['employeeId', 'parentId']]
chart_df = chart_df.dropna()
chart_df

,employeeId,parentId
0,1439351,1439346
1,1439347,1439346
2,1439348,1439346
4,1198798,1198795
5,1198802,1198795
...,...,...
70329,1679508,1679503
70330,1679524,1679503
70331,1679522,1679503
70332,1679515,1679503


In [79]:
chart_df.to_csv(f"orgchart_{NAME}.csv")